In [1]:
import numpy

from cicada.additive import AdditiveProtocolSuite
from cicada.communicator import SocketCommunicator
from cicada.encoding import FixedPoint

def main(communicator):
    protocol = AdditiveProtocolSuite(communicator, order=127, encoding=FixedPoint(precision=2))
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    c_share = protocol.add(a_share, b_share)
    c = protocol.reveal(c_share)

Note that we're using a very small field (`order=127`) and a very low-precision encoding (`precision=2`) to make the values in the logged output easier to read.

In [2]:
import logging

from cicada import transcript

with transcript.record():
    transcript.set_handler(logging.getLogger(), transcript.net_handler())
    SocketCommunicator.run(fn=main, world_size=3);

Player 0: --> 1 PRZS 6351276413306923792
Player 1: --> 2 PRZS 6671259479083241728
Player 2: --> 0 PRZS 6117738161174538279
Player 1: <-- 0 PRZS 6351276413306923792
Player 0: <-- 2 PRZS 6117738161174538279
Player 2: <-- 1 PRZS 6671259479083241728
Player 2: --> 0 GATHER 73
Player 1: --> 0 GATHER 49
Player 0: --> 0 GATHER 25
Player 1: --> 1 GATHER 49
Player 2: --> 1 GATHER 73
Player 0: <-- 1 GATHER 49
Player 0: <-- 2 GATHER 73
Player 1: <-- 1 GATHER 49
Player 2: --> 2 GATHER 73
Player 0: <-- 0 GATHER 25
Player 1: <-- 2 GATHER 73
Player 2: <-- 2 GATHER 73
Player 0: --> 1 GATHER 25
Player 0: --> 2 GATHER 25
Player 1: <-- 0 GATHER 25
Player 2: <-- 0 GATHER 25
Player 1: --> 2 GATHER 49
Player 2: <-- 1 GATHER 49


In [3]:
def main(communicator):
    transcript.log("Let's setup additive sharing!")
    protocol = AdditiveProtocolSuite(communicator, order=127, encoding=FixedPoint(precision=2))
    transcript.log("Let's share some secrets!")
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    transcript.log("Let's add some secrets!")
    c_share = protocol.add(a_share, b_share)
    transcript.log("Let's reveal the results!")
    c = protocol.reveal(c_share)

with transcript.record():
    transcript.set_handler(logging.getLogger(), transcript.net_handler())
    SocketCommunicator.run(fn=main, world_size=3);

Player 0: Let's setup additive sharing!
Player 1: Let's setup additive sharing!
Player 2: Let's setup additive sharing!
Player 0: --> 1 PRZS 7981064255932956492
Player 1: --> 2 PRZS 8997042681810327972
Player 2: --> 0 PRZS 6186966104298353016
Player 1: <-- 0 PRZS 7981064255932956492
Player 2: <-- 1 PRZS 8997042681810327972
Player 0: <-- 2 PRZS 6186966104298353016
Player 1: Let's share some secrets!
Player 0: Let's share some secrets!
Player 2: Let's share some secrets!
Player 1: Let's add some secrets!
Player 2: Let's add some secrets!
Player 0: Let's add some secrets!
Player 1: Let's reveal the results!
Player 2: Let's reveal the results!
Player 0: Let's reveal the results!
Player 1: --> 0 GATHER 118
Player 2: --> 0 GATHER 12
Player 0: --> 0 GATHER 17
Player 1: --> 1 GATHER 118
Player 2: --> 1 GATHER 12
Player 0: <-- 1 GATHER 118
Player 1: <-- 1 GATHER 118
Player 2: --> 2 GATHER 12
Player 0: <-- 2 GATHER 12
Player 1: <-- 2 GATHER 12
Player 2: <-- 2 GATHER 12
Player 0: <-- 0 GATHER 17


In [4]:
with transcript.record():
    transcript.set_handler(logging.getLogger(), transcript.net_handler(fmt="# {processName}: {msg}", netfmt="{processName},{net.verb},{net.src},{net.dst},{net.tag},{net.payload}"))
    SocketCommunicator.run(fn=main, world_size=3);

# Player 0: Let's setup additive sharing!
# Player 2: Let's setup additive sharing!
# Player 1: Let's setup additive sharing!
Player 0,sent,0,1,PRZS,9095744799790441171
Player 1,sent,1,2,PRZS,7680671400645759937
Player 2,sent,2,0,PRZS,5019477953993648987
Player 1,received,0,1,PRZS,9095744799790441171
Player 2,received,1,2,PRZS,7680671400645759937
Player 0,received,2,0,PRZS,5019477953993648987
# Player 1: Let's share some secrets!
# Player 2: Let's share some secrets!
# Player 0: Let's share some secrets!
# Player 2: Let's add some secrets!
# Player 1: Let's add some secrets!
# Player 0: Let's add some secrets!
# Player 2: Let's reveal the results!
# Player 1: Let's reveal the results!
# Player 0: Let's reveal the results!
Player 1,sent,1,0,GATHER,39
Player 2,sent,2,0,GATHER,13
Player 0,sent,0,0,GATHER,95
Player 2,sent,2,1,GATHER,13
Player 1,sent,1,1,GATHER,39
Player 0,received,1,0,GATHER,39
Player 2,sent,2,2,GATHER,13
Player 1,received,2,1,GATHER,13
Player 0,received,2,0,GATHER,13
Play

If you look carefully, you can see that this produces two nearly identical events for each message (once when the message is sent, and once when the message is received).  If you wish to eliminate the duplication, e.g. by only logging messages when they're sent, you can specify that too:

In [5]:
with transcript.record():
    transcript.set_handler(logging.getLogger(), transcript.net_handler(received=False, fmt="# {processName}: {msg}", netfmt="{processName},{net.verb},{net.src},{net.dst},{net.tag},{net.payload}"))
    SocketCommunicator.run(fn=main, world_size=3);

# Player 0: Let's setup additive sharing!
# Player 2: Let's setup additive sharing!
# Player 1: Let's setup additive sharing!
Player 0,sent,0,1,PRZS,6840397465782473542
Player 2,sent,2,0,PRZS,2438917759966484306
Player 1,sent,1,2,PRZS,9076403204914907708
# Player 0: Let's share some secrets!
# Player 1: Let's share some secrets!
# Player 2: Let's share some secrets!
# Player 0: Let's add some secrets!
# Player 2: Let's add some secrets!
# Player 1: Let's add some secrets!
# Player 0: Let's reveal the results!
# Player 2: Let's reveal the results!
# Player 1: Let's reveal the results!
Player 0,sent,0,0,GATHER,4
Player 2,sent,2,0,GATHER,61
Player 1,sent,1,0,GATHER,82
Player 2,sent,2,1,GATHER,61
Player 1,sent,1,1,GATHER,82
Player 0,sent,0,1,GATHER,4
Player 2,sent,2,2,GATHER,61
Player 0,sent,0,2,GATHER,4
Player 1,sent,1,2,GATHER,82


In [6]:
from cicada.arithmetic import Field

with transcript.record():
    transcript.set_handler(logging.getLogger(), transcript.code_handler())
    
    f = Field(order=127)
    a = f.ones(3)
    b = f.uniform(size=3, generator=numpy.random.default_rng())
    f.inplace_add(a, b)

cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).ones(shape=3), numpy.array([1, 1, 1], dtype='object'))

bg = numpy.random.PCG64()
bg.state = {'bit_generator': 'PCG64', 'state': {'state': 151445935697576457205045554731946389149, 'inc': 207607417564250072824718168353264128549}, 'has_uint32': 0, 'uinteger': 0}
cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).uniform(size=3, generator=numpy.random.Generator(bg)), numpy.array([19, 105, 56], dtype='object'))

lhs = numpy.array([1, 1, 1], dtype='object')
cicada.arithmetic.Field(order=127).inplace_add(lhs=lhs, rhs=numpy.array([19, 105, 56], dtype='object'))
cicada.transcript.assert_equal(lhs, numpy.array([20, 106, 57], dtype='object'))



In [7]:
import io

with transcript.record():
    buffer = io.StringIO()
    transcript.set_handler(logging.getLogger(), transcript.code_handler(logging.StreamHandler(buffer)))
    
    f = Field(order=127)
    a = f.ones(3)
    b = f.uniform(size=3, generator=numpy.random.default_rng())
    f.inplace_add(a, b)

In [8]:
buffer.seek(0)
for line in buffer:
    print(line.strip())

cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).ones(shape=3), numpy.array([1, 1, 1], dtype='object'))

bg = numpy.random.PCG64()
bg.state = {'bit_generator': 'PCG64', 'state': {'state': 187213837815921943323857019383963100791, 'inc': 181072851920885976333708426999942580415}, 'has_uint32': 0, 'uinteger': 0}
cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).uniform(size=3, generator=numpy.random.Generator(bg)), numpy.array([40, 61, 91], dtype='object'))

lhs = numpy.array([1, 1, 1], dtype='object')
cicada.arithmetic.Field(order=127).inplace_add(lhs=lhs, rhs=numpy.array([40, 61, 91], dtype='object'))
cicada.transcript.assert_equal(lhs, numpy.array([41, 62, 92], dtype='object'))



Now, we can execute the buffered transcript, one-line-at-a-time, and confirm that none of the assertions raise exceptions:

In [9]:
import cicada.transcript

try:
    buffer.seek(0)
    for line in buffer:
        exec(line)
except Exception as e:
    print(f"Consistency verification failed: {e}")
else:
    print("Consistency verification succeeded.")

Consistency verification succeeded.


Let's go back to our original example. Because it requires three players, we'll setup Cicada to write a separate transcript for each player:  

In [10]:
def main(comm):
    transcript.set_handler(logging.getLogger(), transcript.code_handler(logging.FileHandler(f"player-{comm.rank}.py", "w")))

    transcript.log("Let's setup additive sharing!")
    protocol = AdditiveProtocolSuite(comm, order=127, encoding=FixedPoint(precision=2))
    transcript.log("Let's share some secrets!")
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    transcript.log("Let's add some secrets!")
    c_share = protocol.add(a_share, b_share)
    transcript.log("Let's reveal the results!")
    c = protocol.reveal(c_share)

with transcript.record():
    SocketCommunicator.run(fn=main, world_size=3);

Note that we're configuring the transcript logging inside `main`, since that allows us to generate a filename based on the player rank. Also note that we've left our context messages in-place. Let's see what the results look like for one player:

In [11]:
with open("player-0.py") as stream:
    for line in stream:
        print(line.strip())

# Player 0: Let's setup additive sharing!
# Player 0: Let's share some secrets!
bg = numpy.random.PCG64()
bg.state = {'bit_generator': 'PCG64', 'state': {'state': 267254325009661470179387868466142940374, 'inc': 9635894540038665048287263072219904259}, 'has_uint32': 0, 'uinteger': 0}
cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).uniform(size=(), generator=numpy.random.Generator(bg)), numpy.array(25, dtype='object'))

bg = numpy.random.PCG64()
bg.state = {'bit_generator': 'PCG64', 'state': {'state': 335225335436544017633986842393542797678, 'inc': 65291376465279623857757526391652578125}, 'has_uint32': 0, 'uinteger': 0}
cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).uniform(size=(), generator=numpy.random.Generator(bg)), numpy.array(10, dtype='object'))

lhs = numpy.array(25, dtype='object')
cicada.arithmetic.Field(order=127).inplace_subtract(lhs=lhs, rhs=numpy.array(10, dtype='object'))
cicada.transcript.assert_equal(lhs, numpy.array(15, dtype='object

In [12]:
import traceback

for rank in range(3):
    try:
        with open(f"player-{rank}.py") as stream:
            for line in stream:
                exec(line)
    except Exception as e:
        print(f"Player {rank} consistency verification failed: {e}")
        print(traceback.format_exc())
    else:
        print(f"Player {rank} consistency verification succeeded.")

Player 0 consistency verification succeeded.
Player 1 consistency verification succeeded.
Player 2 consistency verification succeeded.
